# BIPARTITE GRAPH

In [2]:
import pyarrow.parquet as pq


PARQUET_BUSINESS = '../data/parquet/business.parquet'

table = pq.ParquetFile(PARQUET_BUSINESS)
print(table.num_row_groups)

64


In [1]:
import pandas as pd

In [2]:
COLS_PARTNERS = [
    0, 2, 5
]
COLS_COMPANIES = [
    0, 1, 4
]
COLS_BUSINESS = [
    0, 1, 2, 3, 4, 6, 10, 18
]

NAMES_PARTNERS  = [
    'cnpj'        ,
    'name_partner',
    'start_date'  ,
]
NAMES_COMPANIES = [
    'cnpj'          ,
    'corporate_name',
    'capital'       ,
]
NAMES_BUSINESS  = [
    'cnpj'        ,
    'cnpj_order'  ,
    'cnpj_dv'     ,
    'branch'      ,
    'trade_name'  ,
    'closing_date',
    'opening_date',
    'cep'         ,
]

In [ ]:
import pandas as pd


estabelecimentos = pd.read_parquet(
    '../data/parquet/companies.parquet',
    columns=['cnpj'],
)

estabelecimentos.head()

NameError: name 'pd' is not defined

In [11]:
# Verifica se todos os CNPJs de partners estão em companies usando DuckDB
query_companies = """
    SELECT COUNT(*) = 0 AS todos_em_companies
    FROM (
        SELECT cnpj
        FROM '../data/parquet/partners.parquet'
        EXCEPT
        SELECT cnpj
        FROM '../data/parquet/companies.parquet'
    )
"""
todos_em_companies = con.execute(query_companies).fetchdf().iloc[0, 0]
print('Todos os CNPJs de partners estão em companies?', bool(todos_em_companies))

# Verifica se todos os CNPJs de partners estão em business usando DuckDB
query_business = """
    SELECT COUNT(*) = 0 AS todos_em_business
    FROM (
        SELECT cnpj
        FROM '../data/parquet/partners.parquet'
        EXCEPT
        SELECT cnpj
        FROM '../data/parquet/business.parquet'
    )
"""
todos_em_business = con.execute(query_business).fetchdf().iloc[0, 0]
print('Todos os CNPJs de partners estão em business?', bool(todos_em_business))

Todos os CNPJs de partners estão em companies? True
Todos os CNPJs de partners estão em business? True


In [1]:
import duckdb

# Conecta ao arquivo Parquet usando DuckDB
con = duckdb.connect()

# Conta o número total de linhas e o número de linhas únicas de 'estabelecimentos'
result = con.execute("""
    SELECT 
        COUNT(cnpj) AS total_linhas,
        COUNT(DISTINCT cnpj) AS linhas_unicas
    FROM '../data/parquet/partners.parquet'
""").fetchdf()

print(result)

   total_linhas  linhas_unicas
0      26060365       15155173


In [2]:
import duckdb

# Conecta ao arquivo Parquet usando DuckDB
con = duckdb.connect()

# Conta o número total de linhas e o número de linhas únicas de 'estabelecimentos'
result = con.execute("""
    SELECT 
        COUNT(cnpj) AS total_linhas,
        COUNT(DISTINCT cnpj) AS linhas_unicas
    FROM '../data/parquet/companies.parquet'
""").fetchdf()

print(result)

   total_linhas  linhas_unicas
0      15155173       15155173


In [7]:
result = con.execute("""
    SELECT 
        COUNT(*) AS total_linhas,
        COUNT(DISTINCT (cnpj, cnpj_order, cnpj_dv)) AS linhas_unicas,
        COUNT(DISTINCT cnpj) AS cnpjs_unicas
    FROM '../data/parquet/business.parquet'
""").fetchdf()

print(result)

   total_linhas  linhas_unicas  cnpjs_unicas
0      17520764       17520764      15155173


In [6]:
con = duckdb.connect()

# Conta o número total de linhas e o número de linhas únicas de 'estabelecimentos'
result = con.execute("""
    SELECT 
        COUNT((cnpj, cnpj_order, cnpj_dv)) AS total_linhas,
        COUNT(DISTINCT (cnpj, cnpj_order, cnpj_dv)) AS linhas_unicas
    FROM '../data/parquet/business.parquet'
""").fetchdf()

print(result)

   total_linhas  linhas_unicas
0      17520764       17520764


In [19]:
df = pd.read_csv('../data/csv/estabelecimentos/estabelecimentos0.csv', sep=';', usecols=COLS_BUSINESS, names=NAMES_BUSINESS, nrows=1000000,
                 low_memory=False, encoding='latin-1', on_bad_lines='skip')

df.head()

,cnpj,cnpj_order,cnpj_dv,branch,trade_name,closing_date,opening_date,cep
0,20653224,1,0,1,ASSEMBLEIA DE DEUS CATEDRAL DOS EVANGELISTAS,20181213,20140714,68626414.0
1,20653240,1,0,1,ATELLIERDABELEZZA P7 COM BR,20160830,20140717,3420000.0
2,20653251,1,82,1,NaN,20140717,20140717,40231181.0
3,20653263,1,7,1,SANDRA DAICAMPI,20150928,20140717,88270000.0
4,20653277,1,20,1,MARQUINHOS RACOES,20160511,20140717,27961050.0


In [20]:
df.cep = (
    df
    .cep
    .fillna('0')
    .astype(str)
    .str.replace('-' , '')
    .str.replace(r'\..*$', '', regex=True)
    .str.zfill(8)
)

In [21]:
df

,cnpj,cnpj_order,cnpj_dv,branch,trade_name,closing_date,opening_date,cep
0,20653224,1,0,1,ASSEMBLEIA DE DEUS CATEDRAL DOS EVANGELISTAS,20181213,20140714,68626414
1,20653240,1,0,1,ATELLIERDABELEZZA P7 COM BR,20160830,20140717,03420000
2,20653251,1,82,1,NaN,20140717,20140717,40231181
3,20653263,1,7,1,SANDRA DAICAMPI,20150928,20140717,88270000
4,20653277,1,20,1,MARQUINHOS RACOES,20160511,20140717,27961050
...,...,...,...,...,...,...,...,...
999995,30609949,1,54,1,NaN,20210325,20180604,21550220
999996,30609961,1,69,1,DINO BRINDES,20180604,20180604,20740480
999997,30609974,1,38,1,NaN,20180604,20180604,15046826
999998,30609987,1,7,1,NaN,20241105,20180604,38405322


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 3 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   cnpj          1000000 non-null  int64 
 1   name_partner  999816 non-null   object
 2   start_date    1000000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 22.9+ MB


In [22]:
OUTPUT_PARTNERS  = '../data/parquet/partners.parquet'
OUTPUT_COMPANIES = '../data/parquet/companies.parquet'
OUTPUT_BUSINESS  = '../data/parquet/business.parquet'

In [23]:
import fastparquet as fp


In [26]:
pf = fp.ParquetFile(OUTPUT_COMPANIES)
pf.head(10)

,cnpj,corporate_name,capital
0,41273590,MARIA DAS MERCES SOARES LEMOS,0
1,41273608,FRANCISCA SAMPAIO DE OLIVEIRA,0
2,41273616,ANTONIO MORAIS DA SILVA,0
3,41273624,FRANCISCA MIRANDA OLIVEIRA,0
4,41273632,C NASCIMENTO AQUINO,0
5,41273657,R C DE OLIVEIRA LIMA,0
6,41273665,NESTOR FERREIRA DA SILVA,0
7,41273673,ANTONIO P DA SILVA MERCEARIA,0
8,41273681,LUIZ DOS SANTOS COELHO,0
9,41273683,GUILHERME DO VALE MORAES,0


In [6]:
chunk = pd.read_csv(
    '../data/csv/empresas/empresas0.csv',
    sep=';',
    usecols=[4],
    names=['capital'],
    nrows=1000000,
    low_memory=False      ,
    encoding='latin-1'    ,
    on_bad_lines='skip'   ,
)

chunk.capital = (
    chunk.capital
    .str
    .replace(r',.*', '', regex=True)
    .astype('int64')
)

In [21]:
import pyarrow.parquet as pq

parquet_file = pq.ParquetFile(OUTPUT_BUSINESS)

table = parquet_file.read_row_group(0)
table.to_pandas().head(10)

,cnpj,cnpj_order,cnpj_dv,branch,trade_name,closing_date,opening_date,cep
0,29987225,0001,65,True,None,1903-12-11,1903-12-11,99200000.0
1,22244392,0001,77,True,None,1935-12-31,1935-12-31,95320000.0
2,21479204,0001,27,True,COMUNIDADE EVANGELICA DE NOVA BREMEN,1941-12-15,1941-12-15,89140000.0
3,21553560,0001,43,True,None,1947-08-21,1947-08-21,22440060.0
4,31971844,0001,03,True,None,1952-09-17,1952-09-17,22430190.0
5,27840805,0001,08,True,None,1953-11-05,1953-11-05,5652030.0
6,29473461,0001,63,True,CMARA DE VEREADORES DE ARATIBA/RS,1955-10-04,1955-10-04,99770000.0
7,22739620,0001,80,True,None,1957-01-05,1957-01-05,90460140.0
8,30012840,0001,35,True,PODER LEGISLATIVO DE SEBERI,1959-01-31,1959-01-31,98380000.0
9,26095234,0001,53,True,None,1961-02-09,1961-02-09,4052020.0


In [18]:
parquet_file.schema

required group field_id=-1 schema {
  optional binary field_id=-1 cnpj (String);
  optional binary field_id=-1 cnpj_order (String);
  optional binary field_id=-1 cnpj_dv (String);
  optional boolean field_id=-1 branch;
  optional binary field_id=-1 trade_name (String);
  optional int64 field_id=-1 closing_date (Timestamp(isAdjustedToUTC=false, timeUnit=nanoseconds, is_from_converted_type=false, force_set_converted_type=false));
  optional int64 field_id=-1 opening_date (Timestamp(isAdjustedToUTC=false, timeUnit=nanoseconds, is_from_converted_type=false, force_set_converted_type=false));
  optional binary field_id=-1 cep (String);
}

In [3]:
import pyarrow.parquet as pq
import pandas as pd


parquet_file = pq.ParquetFile(OUTPUT_COMPANIES)

print(f'Total de row_groups: {parquet_file.num_row_groups}')

duplicados_por_rg = []
for i in range(parquet_file.num_row_groups):
    print(f'Processando row_group {i}...')

    table = parquet_file.read_row_group(i, columns=['cnpj'])
    df = table.to_pandas()

    repetidos = df[df.duplicated(subset='cnpj', keep=False)]

    if not repetidos.empty:
        repetidos['row_group'] = i
        duplicados_por_rg.append(repetidos)

if duplicados_por_rg:
    repetidos_final = pd.concat(duplicados_por_rg, ignore_index=True)
    print('Duplicados dentro dos row_groups encontrados:')
    print(repetidos_final)
else:
    print('Nenhum CNPJ duplicado dentro dos row_groups foi encontrado.')


Total de row_groups: 13
Processando row_group 0...
Processando row_group 1...
Processando row_group 2...
Processando row_group 3...
Processando row_group 4...
Processando row_group 5...
Processando row_group 6...


/tmp/ipykernel_6263/3132764133.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repetidos['row_group'] = i


Processando row_group 7...
Processando row_group 8...
Processando row_group 9...
Processando row_group 10...
Processando row_group 11...
Processando row_group 12...
Duplicados dentro dos row_groups encontrados:
       cnpj  row_group
0  10959550          6
1  10959550          6
2  10959550          6


In [3]:
business = pd.read_parquet(OUTPUT_BUSINESS, columns=['trade_name'])

print(business[business.trade_name.notnull()].sample(3))

                                  trade_name
25793726  EMPREITEIRA DE MAO DE OBRA ESTRELA
5409036                  CARDOSO IMOBILIARIA
12401990          LATICINIO SITIO MANGUEIRAL


In [7]:
import pyarrow.parquet as pq
import pyarrow.compute as pc


table = pq.read_table(OUTPUT_BUSINESS, columns=['trade_name'])

# Aplica filtro para linhas onde trade_name não é nulo
filtered_table = table.filter(pc.field('trade_name').is_valid())

# Converte para pandas DataFrame
df_filtered = filtered_table.to_pandas().sample(3)

print(df_filtered)


                     trade_name
22283709        GARDENIAQUEIROZ
19473901   COMERCIAL CARANGUEJO
10120327  ESTAMPARIA SO VITORIA


In [17]:
df_filtered.trade_name

22283709          GARDENIAQUEIROZ
19473901     COMERCIAL CARANGUEJO
10120327    ESTAMPARIA SO VITORIA
Name: trade_name, dtype: object

In [4]:
business.max()

cnpj_order    9999
dtype: int16

In [10]:
len(repetidos)

3

In [ ]:
duplicated_cnpjs = companies[companies.duplicated(subset='cnpj', keep=False)]['cnpj']

print(duplicated_cnpjs)

In [15]:
partners[partners.name_partner.str.len() < 3]

,cnpj,name_partner,start_date
8,2450527,!,19010101
2048975,2327780,S,19980109


In [ ]:
# ==============================================================
# PART 4 - Reload both files and rewrite with higher compression
# ==============================================================
print('Rewriting parquet files with pyarrow compression...')


partners.sort_values(
    by=[
        'start_date'  ,
        'name_partner',
    ],
    inplace=True,
)
partners.to_parquet(
    OUTPUT_PARTNERS ,
    engine='pyarrow',
    index=False     ,
)

del partners


business = pd.read_parquet(OUTPUT_BUSINESS)

business.sort_values(
    by=[
        'closing_date',
        'opening_date',
        'cep'         ,
    ],
    inplace=True,
)
business.to_parquet(
    OUTPUT_BUSINESS ,
    engine='pyarrow',
    index=False     ,
)

del business


## PREPARING THE ENVIRONMENT

## BIPARTITE GRAPH

Generating the bipartite graph:

In [3]:
cnpj2id    = {}
partner2id = {}
adjdict    = defaultdict(list)

cnpj_id  = 0
cnpj_idx = 0
partner_id  = 0
partner_idx = 0

for idx, (cnpj, partner) in enumerate(partners.itertuples(index=False)):
    if cnpj not in cnpj2id:
        cnpj_idx  = cnpj_id
        cnpj_id  += 1

        cnpj2id[cnpj] = cnpj_idx
    else:
        cnpj_idx = cnpj2id[cnpj]

    if partner not in partner2id:
        partner_idx  = partner_id
        partner_id  += 1

        partner2id[partner] = partner_idx
    else:
        partner_idx = partner2id[partner]

    adjdict[cnpj_idx].append(partner_idx)

    if idx % 1000000 == 999999:
        print('Number of rows processed =', idx + 1)

Number of rows processed = 1000000
Number of rows processed = 2000000
Number of rows processed = 3000000
Number of rows processed = 4000000
Number of rows processed = 5000000
Number of rows processed = 6000000
Number of rows processed = 7000000
Number of rows processed = 8000000
Number of rows processed = 9000000
Number of rows processed = 10000000
Number of rows processed = 11000000
Number of rows processed = 12000000
Number of rows processed = 13000000
Number of rows processed = 14000000
Number of rows processed = 15000000
Number of rows processed = 16000000
Number of rows processed = 17000000
Number of rows processed = 18000000
Number of rows processed = 19000000
Number of rows processed = 20000000
Number of rows processed = 21000000
Number of rows processed = 22000000
Number of rows processed = 23000000
Number of rows processed = 24000000
Number of rows processed = 25000000


Optimizing the data structure of the adjacency list:

In [4]:
adjacency = [[]] * len(adjdict)

for k, v in adjdict.items():
    adjacency[k] = v

del adjdict

Saving the data:

In [5]:
with open('../data/cnpj2id.pkl'   , 'wb') as f:
    pickle.dump(cnpj2id   , f)

with open('../data/partner2id.pkl', 'wb') as f:
    pickle.dump(partner2id, f)

with open('../data/adjacency.pkl' , 'wb') as f:
    pickle.dump(adjacency , f)

Freeing the memory of objects:

In [6]:
del cnpj2id
del partner2id
del adjacency

## EXPLORING THE RESULTS

Checking the number of companies and partners:

In [7]:
print('Number of companies in the \'partners\' file =', partners.cnpj.nunique())
print('Number of partners  in the \'partners\' file =', partners.name_partner.nunique())

Number of companies in the 'partners' file = 14441622
Number of partners  in the 'partners' file = 14686766


Freeing up resources:

In [8]:
del partners

Loading the data:

In [9]:
with open('../data/cnpj2id.pkl'   , 'rb') as f:
    cnpj2id = pickle.load(f)

with open('../data/partner2id.pkl', 'rb') as f:
    partner2id = pickle.load(f)

with open('../data/adjacency.pkl' , 'rb') as f:
    adjacency = pickle.load(f)

Confirming the number of companies and partners:

In [10]:
print('Number of companies in the \'cnpj2id\'    file =', len(cnpj2id   ))
print('Number of partners  in the \'partner2id\' file =', len(partner2id))

Number of companies in the 'cnpj2id'    file = 14441622
Number of partners  in the 'partner2id' file = 14686766


Obtaining the partners of a given company:

In [11]:
CNPJ = 85061877


idx      = cnpj2id[CNPJ]
partners = set(adjacency[idx])

print(f'CNPJ members {CNPJ}:')
print()
for nome, idx in partner2id.items():
    if idx in partners:
        partners.remove(idx)

        print(nome)

    if not partners:
        break

CNPJ members 85061877:

RICARDO BAULE ROSSI
EDUARDO GUILHERME BAULE ROSSI
